In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# GPU selection (memory)
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# Data info.

width = 640
height = 480
channel = 1


# resize
r_w = 512
r_h = 512

total_pix = r_w * r_h * channel

In [4]:
data_txt = open('./data_split/data_dir_dict_0614.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [5]:
ev_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
ev_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

for i in range(len(tr_data_dir['test'])):
    img = cv2.imread(tr_data_dir['test'][i][0],-1)
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    ev_data[i,:] = img2.flatten()
    ev_label[i] = tr_data_dir['test'][i][1]

In [6]:
f_dir1 = '/models/Edge/Train_0/model.ckpt-2280.meta'
f_dir2 = '/models/Edge/Train_0/'

saver = tf.train.import_meta_graph(f_dir1)
batch = 32
ev_steps = int(ev_label.shape[0]/batch)
ev_acc = np.zeros(ev_label.shape[0])

with tf.Session() as sess:
    
    new_saver = tf.train.import_meta_graph(f_dir1)
    new_saver.restore(sess, tf.train.latest_checkpoint(f_dir2))
    graph = tf.get_default_graph()
    #for op in tf.get_default_graph().get_operations():
        #print(op.name)
        #print(op.values())
        
    data_in = graph.get_tensor_by_name("fifo_queue_DequeueUpTo:1")

    #feed_dict={data_in:ev_data[0:10], label:ev_label[0:10]}
    keep_prob = graph.get_tensor_by_name("dropout/dropout/keep_prob:0")
    for i in range(ev_steps):
        if i < (ev_steps-1):
            feed_dict={data_in:ev_data[batch*i:batch*(i+1)], keep_prob:1.0}
            label = ev_label[batch*i:batch*(i+1)].reshape(batch)
        else:
            feed_dict={data_in:ev_data[batch*i:], keep_prob:1.0}
            temp = ev_label[batch*i:]
            label = temp.reshape(len(temp))
        
        prob = graph.get_tensor_by_name("softmax_tensor:0")
        classes = graph.get_tensor_by_name("ArgMax:0")
        
        
        result_pr = sess.run(prob, feed_dict=feed_dict)
        result_cl = sess.run(classes, feed_dict=feed_dict)
        
        print(result_pr)
        print(result_cl)
        print(label)

        acc = np.equal(result_cl, label).reshape((1,len(label)))*1
        
        print(acc)
        print(np.mean(acc))
        
        if i < (ev_steps-1):
            ev_acc[batch*i:batch*(i+1)] = acc
        else:
            ev_acc[batch*i:] = acc


INFO:tensorflow:Restoring parameters from /models/Edge/Train_0/model.ckpt-2280
[[6.3936582e-06 9.9999356e-01]
 [6.8795313e-03 9.9312055e-01]
 [9.9999988e-01 1.5948854e-07]
 [1.0000000e+00 4.2353873e-13]
 [1.0000000e+00 2.7924253e-08]
 [1.6101800e-04 9.9983895e-01]
 [6.5285695e-07 9.9999940e-01]
 [4.2116961e-01 5.7883042e-01]
 [9.9915922e-01 8.4083446e-04]
 [6.5469384e-23 1.0000000e+00]
 [3.2398311e-05 9.9996758e-01]
 [9.9790561e-01 2.0943624e-03]
 [4.1067442e-06 9.9999595e-01]
 [4.3296558e-03 9.9567032e-01]
 [8.0431048e-09 1.0000000e+00]
 [9.9992752e-01 7.2464383e-05]
 [7.2244522e-13 1.0000000e+00]
 [6.3252389e-14 1.0000000e+00]
 [5.3814300e-03 9.9461854e-01]
 [1.2341139e-05 9.9998760e-01]
 [9.9996364e-01 3.6394453e-05]
 [3.0643110e-05 9.9996936e-01]
 [9.9989998e-01 1.0002517e-04]
 [9.9697000e-01 3.0300135e-03]
 [9.3905754e-02 9.0609425e-01]
 [1.0000000e+00 7.3682144e-11]
 [6.0873138e-09 1.0000000e+00]
 [4.0577626e-05 9.9995947e-01]
 [9.9992013e-01 7.9886486e-05]
 [4.0002913e-05 9.9995

[[4.2262630e-04 9.9957734e-01]
 [9.9999988e-01 9.4230955e-08]
 [9.9997759e-01 2.2428445e-05]
 [1.0612935e-07 9.9999988e-01]
 [1.0000000e+00 1.8478070e-09]
 [9.9999261e-01 7.3619481e-06]
 [1.8169987e-07 9.9999976e-01]
 [1.0000000e+00 4.4456228e-16]
 [7.5385920e-10 1.0000000e+00]
 [5.6590684e-11 1.0000000e+00]
 [9.9996448e-01 3.5545778e-05]
 [9.7686076e-01 2.3139216e-02]
 [9.9976701e-01 2.3297919e-04]
 [8.3430201e-01 1.6569796e-01]
 [7.1953371e-10 1.0000000e+00]
 [9.9999976e-01 1.8151560e-07]
 [9.9676919e-01 3.2308025e-03]
 [8.9215746e-08 9.9999988e-01]
 [9.9999630e-01 3.6567003e-06]
 [9.9999976e-01 2.1079745e-07]
 [8.5593943e-05 9.9991441e-01]
 [9.9260497e-01 7.3950682e-03]
 [1.0000000e+00 5.7017332e-19]
 [3.7976584e-23 1.0000000e+00]
 [1.0000000e+00 2.1333818e-17]
 [1.4235920e-12 1.0000000e+00]
 [9.9998927e-01 1.0758639e-05]
 [1.0000000e+00 5.2193981e-22]
 [4.0315227e-11 1.0000000e+00]
 [9.9999976e-01 2.4485882e-07]
 [9.9981636e-01 1.8366947e-04]
 [1.4658270e-05 9.9998534e-01]]
[1 0 0 

[[9.99999881e-01 1.63597676e-07]
 [4.30900045e-03 9.95691001e-01]
 [9.99954462e-01 4.55466507e-05]
 [1.01717655e-02 9.89828229e-01]
 [9.99986053e-01 1.39571139e-05]
 [9.97721612e-01 2.27840897e-03]
 [6.52958465e-10 1.00000000e+00]
 [4.34049185e-10 1.00000000e+00]
 [9.99926209e-01 7.37444789e-05]
 [1.00000000e+00 2.07156647e-09]
 [1.55380003e-05 9.99984503e-01]
 [9.98304725e-01 1.69526646e-03]
 [5.38773453e-13 1.00000000e+00]
 [9.99967933e-01 3.21132902e-05]
 [1.94330582e-07 9.99999762e-01]
 [9.98347878e-01 1.65214553e-03]
 [1.62093178e-03 9.98379111e-01]
 [8.62419844e-01 1.37580097e-01]
 [9.99588907e-01 4.11098881e-04]
 [3.14437532e-10 1.00000000e+00]
 [2.04962403e-09 1.00000000e+00]
 [1.00000000e+00 4.54643024e-13]
 [7.15686532e-04 9.99284327e-01]
 [9.44672227e-01 5.53277433e-02]
 [9.84887183e-01 1.51128182e-02]
 [1.59360276e-08 1.00000000e+00]
 [2.13689710e-09 1.00000000e+00]
 [6.40469216e-06 9.99993563e-01]
 [6.13223076e-01 3.86776894e-01]
 [9.99989629e-01 1.03278335e-05]
 [1.947477

[[5.4131382e-05 9.9994588e-01]
 [9.9980181e-01 1.9818589e-04]
 [5.2765063e-05 9.9994719e-01]
 [9.9944073e-01 5.5919873e-04]
 [9.9999535e-01 4.5921270e-06]
 [7.3095007e-06 9.9999273e-01]
 [1.0000000e+00 4.0304541e-11]
 [4.7633802e-10 1.0000000e+00]
 [1.0000000e+00 1.3878051e-17]
 [9.9999654e-01 3.4442853e-06]
 [9.9807715e-01 1.9227938e-03]
 [9.9913722e-01 8.6276466e-04]
 [9.9999928e-01 6.9892059e-07]
 [9.9980527e-01 1.9470506e-04]
 [9.9910223e-01 8.9778146e-04]
 [7.2682832e-12 1.0000000e+00]
 [7.6768529e-01 2.3231469e-01]
 [5.5669351e-11 1.0000000e+00]
 [9.8594946e-01 1.4050561e-02]
 [1.0000000e+00 7.7471985e-14]
 [9.9999523e-01 4.7115914e-06]
 [9.9999094e-01 9.0702451e-06]
 [4.1964685e-09 1.0000000e+00]
 [2.6984792e-02 9.7301519e-01]
 [1.0000000e+00 2.9302001e-20]
 [1.0000000e+00 3.1529678e-11]
 [5.7528943e-10 1.0000000e+00]
 [9.9997294e-01 2.7044556e-05]
 [1.0000000e+00 4.9154709e-16]
 [9.9873143e-01 1.2686109e-03]
 [9.9189115e-01 8.1088580e-03]
 [9.1932911e-01 8.0670886e-02]]
[1 0 1 

In [7]:
print(ev_acc[-1])
print(np.mean(ev_acc))

1.0
0.9507658643326039


In [8]:
final = np.array([0.9398, 0.9517, 0.9639, 0.9737, 0.9814, 0.9639, 0.9595, 0.8742, 0.9814, 0.9825])
print(np.mean(final))
print(np.std(final))

0.9572
0.03066056098638773


In [9]:
fail_idx = np.where(ev_acc == 0)
print(fail_idx[0])

[ 24  39  58  62  70  74  80 111 113 115 119 135 180 196 236 259 303 307
 319 364 487 502 528 535 555 593 630 638 639 654 661 674 688 690 695 725
 733 740 752 771 772 825 852 868 904]


In [10]:

for i in range(len(fail_idx[0])):
    new_img = cv2.imread(tr_data_dir['test'][fail_idx[0][i]][0])
    
    save_path = "/models/Edge-fail/T0_fail-%d.png" % i
    cv2.imwrite(save_path, new_img)
